# Register Client and Create Access Token Notebook
- Find detailed information about client registration and access tokens in this blog post: [Authentication to SAS Viya: a couple of approaches](https://blogs.sas.com/content/sgf/2021/09/24/authentication-to-sas-viya/)
- Use the client_id to create an access token you can use in the Jupyter environment or externally for API calls to SAS Viya.
- You must add the following info to the script: client_id, client_secret, baseURL, and consul_token
- Additional access token information is found at the end of this notebook.


### Run the cells below and follow the resulting instructions.

# Get register access token

In [ ]:
import requests
import json
import os
import base64

# set/create variables
client_id=""
client_secret=""
baseURL = "" # sasserver.sas.com
consul_token = ""

# generate API call for register access token
url = f"https://{baseURL}/SASLogon/oauth/clients/consul?callback=false&serviceId={client_id}"
headers = {
'X-Consul-Token': consul_token
}

# process the results
response = requests.request("POST", url, headers=headers, verify=False)

register_access_token = json.loads(response.text)['access_token']
print(json.dumps(response.json(), indent=4, sort_keys=True))

# Register the client

In [ ]:
# create API call payload data
payload='{"client_id": "' + client_id +'","client_secret": "'+ client_secret +'","scope": ["openid", "*"],"authorized_grant_types": ["authorization_code","refresh_token"],"redirect_uri": "urn:ietf:wg:oauth:2.0:oob","access_token_validity": "43199"}'

# generate API call for register access token
url = f"https://{baseURL}/SASLogon/oauth/clients"
headers = {
'Content-Type': 'application/json',
'Authorization': "Bearer " + register_access_token
}

# process the results
response = requests.request("POST", url, headers=headers, data=payload, verify=False)
print(json.dumps(response.json(), indent=4, sort_keys=True))

# Create access token

In [ ]:
# create authorization url
codeURL = "https://" + baseURL + "/SASLogon/oauth/authorize?client_id=" + client_id + "&response_type=code"

# enccode client string
client_string = client_id + ":" + client_secret
message_bytes = client_string.encode('ascii')
base64_bytes = base64.b64encode(message_bytes)
base64_message = base64_bytes.decode('ascii')

# promt with instructions and entry for auth code
print(f"* Please visit the following site {codeURL}")
print("* If provided a login prompt, add your SAS login credentials")
print("* Once authenticated, you'll be redirected to an authoriztion screen, check all of the boxes that appear")
print("* This will result in a short string of numbers and letters such as `VAxVFVEnKr`; this is your authorization code; copy the code")
code = input("Please enter the authoriztion code you generated through the previous instructions, and then press Enter: ")

# generate API call for access token
url = f"https://{baseURL}/SASLogon/oauth/token?grant_type=authorization_code&code={code}"
headers = {
'Accept': 'application/json',
'Content-Type': 'application/x-www-form-urlencoded',
'Authorization': "Basic " + base64_message
}

# process the results
response = requests.request("GET", url, headers=headers, verify=False)
access_token = json.loads(response.text)['access_token']
print(json.dumps(response.json(), indent=4, sort_keys=True))

# Create access_token.txt file 
directory = os.getcwd()
with open(directory + '/access_token.txt', 'w') as f:
    f.write(access_token)
print('The access token was stored for you as ' + directory + '/access_token.txt')

## Notes on the access token
- The access token has a 12 hour time-to-live (ttl) by default.
- The authorization code is good for 30 minutes and is only good for a single use. 
- You can generate a new authorization code by reusing the authorization url.
- The access_token is valid in this Notebook and is transferable to other notebooks and used for external API calls.